In [1]:
import pandas as pd
import numpy as np
import string
import re
import itertools
import collections
import seaborn as sns
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

In [2]:
def read_file(path,lang='rus'):
    with open(path, 'r') as file:
        if lang == 'rus':
            return re.sub(r'[^А-Яа-я ]', '', file.read().lower())
        else:
            return re.sub(r'[^A-za-z ]', '', file.read().lower())

    
def freq_dict_create(text:str, bigrams=False,prep=True):
    if prep:
        text = re.sub(r'[^А-Яа-я ]', '', text.lower())
    norm_del = len(text)
    if bigrams:
        freq_dict = collections.Counter()
        for i in range(len(text) - 1):
            freq_dict[text[i:i+2]] += 1
        norm_del = sum(freq_dict.values())
    else:
        freq_dict = collections.Counter(text)
    freq_dict = {k: v / norm_del for k,v in sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)}
    return freq_dict


def plot_freq(freq_dict: dict,grid = False):
    labels, values = zip(*freq_dict.items())
    indexes = np.arange(len(labels))
    width = 1
    plt.figure(figsize=(20,15))
    plt.bar(indexes, values, width)
    plt.xticks(indexes + width * 0.5, labels)
    plt.title("Частота букв для Войны и МИР на английском")
    plt.ylabel("Частота")
    plt.xlabel("Символ")
    if grid:
        plt.grid(True)
#     plt.xticks(rotation='vertical')
    plt.show()
    
def create_ed_dict(freq_dict:dict):
    tmp = list(freq_dict)
    random.shuffle(tmp)
    return {k:v for k, v in zip(freq_dict,tmp)}, {v:k for k, v in zip(freq_dict,tmp)}

def encoder(text:str,bigrams=False,prep=True):
    if prep:
        text = re.sub(r'[^А-Яа-я ]', '', text.lower())
    encoder_dict = freq_dict_create(text, bigrams)
    tmp = list(encoder_dict.keys())
    random.shuffle(tmp)
    encoder_dict = {k:v for k,v in zip(encoder_dict, tmp)}
    if bigrams:
        res = ''.join([encoder_dict[i] for i in ["".join((v, g)) for v,g in zip(text[::2], text[1::2])]])
    else:
        res = ''.join([encoder_dict[i] for i in text])
    return res
    
def decoder(text: str, orig_dict: dict,bigrams=False,prep=True):
    if prep:
        text = re.sub(r'[^А-Яа-я ]', '', text.lower())
    freq_dict = freq_dict_create(text,bigrams)
    decoder_dict = {k:v for k, v in zip(freq_dict, orig_dict)}

    if bigrams:
        res = ''.join([decoder_dict[i] for i in ["".join((v, g)) for v,g in zip(text[::2], text[1::2])]])
    else:
        res = ''.join([decoder_dict[i] for i in text])
    return res

In [3]:
anna_karenina = read_file('corpora/AnnaKarenina.txt')
war_peace = read_file('corpora/WarAndPeace.txt')
war_peace_en = read_file('corpora/WarAndPeaceEng.txt',lang='eng')

# Часть 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:  
подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);  
возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;  
расшифруйте их таким частотным методом.


In [4]:
freq_dict_anna = freq_dict_create(anna_karenina)
freq_dict_war = freq_dict_create(war_peace)
freq_dict_war_en = freq_dict_create(war_peace_en)

Для теста используем отрываок из собачьего сердца

In [5]:
test_text = """Мелеховский двор - на самом краю хутора.  Воротца  со  скотиньего  база
ведут на  север  к  Дону.  Крутой  восьмисаженный  спуск  меж  замшелых  в
прозелени меловых глыб, и вот берег: перламутровая россыпь ракушек,  серая
изломистая кайма нацелованной волнами гальки и дальше -  перекипающее  под
ветром вороненой рябью стремя Дона. На  восток,  за  красноталом  гуменных
плетней,  -  Гетманский  шлях,  полынная  проседь,  истоптанный   конскими
копытами бурый, живущий придорожник,  часовенка  на  развилке;  за  ней  -
задернутая текучим маревом степь. С юга - меловая хребтина горы. На  запад
- улица, пронизывающая площадь, бегущая к займищу.
   В предпоследнюю  турецкую  кампанию  вернулся  в  хутор  казак  Мелехов
Прокофий. Из Туретчины привел он  жену  -  маленькую,  закутанную  в  шаль
женщину. Она прятала лицо, редко показывая тоскующие одичалые глаза. Пахла
шелковая шаль далекими неведомыми запахами, радужные узоры ее питали бабью
зависть. Пленная турчанка сторонилась родных Прокофия,  и  старик  Мелехов
вскоре отделил сына. В курень его не ходил до смерти, не забывая обиды.
   Прокофий обстроился скоро: плотники срубили курень, сам пригородил базы
для скотины и к осени увел на новое хозяйство  сгорбленную  иноземку-жену.
Шел с ней за арбой с имуществом по хутору - высыпали на улицу все, от мала
до велика. Казаки сдержанно посмеивались в бороды, голосисто перекликались
бабы, орда немытых казачат улюлюкала  Прокофию  вслед,  но  он,  распахнув
чекмень, шел медленно, как по пахотной борозде,  сжимал  в  черной  ладони
хрупкую кисть жениной руки, непокорно нес белесо-чубатую  голову,  -  лишь
под скулами у него  пухли  и  катались  желваки  да  промеж  каменных,  по
всегдашней неподвижности, бровей проступил пот.
   С той поры редко видели его в хуторе, не бывал он и на майдане.  Жил  в
своем курене, на отшибе у Дона,  бирюком.  Гутарили  про  него  по  хутору
чудное. Ребятишки, пасшие за прогоном телят,  рассказывали,  будто  видели
они, как Прокофий вечерами, когда вянут  зори,  на  руках  носил  жену  до
Татарского, ажник, кургана. Сажал ее там  на  макушке  кургана,  спиной  к
источенному столетиями ноздреватому камню, садился  с  ней  рядом,  и  так
подолгу глядели они в степь. Глядели до тех пор,  пока  истухала  заря,  а
потом Прокофий кутал жену в зипун и на руках относил домой. Хутор  терялся
в догадках, подыскивая объяснение таким  диковинным  поступкам,  бабам  за
разговорами поискаться некогда было. Разно гутарили  и  о  жене  Прокофия:
одни утверждали, что красоты она досель  невиданной,  другие  -  наоборот.
Решилось все после того, как  самая  отчаянная  из  баб,  жалмерка  Мавра,
сбегала к Прокофию  будто  бы  за  свежей  накваской.  Прокофий  полез  за
накваской в погреб, а за  это  время  Мавра  и  разглядела,  что  турчанка
попалась Прокофию последняя из никудышных..."""

In [6]:
print("Оригинальный текст:\n",
      test_text, '\n\n'
      "Закодированый текст:\n",
      encoder(test_text), '\n\n'
      "Декодированый текст:\n",
      decoder(encoder(test_text), freq_dict_anna), '\n\n',
      "Оценка:\n",
      sum(i == j for i,j in zip(re.sub(r'[^А-Яа-я ]', '', test_text.lower()),decoder(encoder(test_text), freq_dict_anna)))/ len(test_text)
     )

Оригинальный текст:
 Мелеховский двор - на самом краю хутора.  Воротца  со  скотиньего  база
ведут на  север  к  Дону.  Крутой  восьмисаженный  спуск  меж  замшелых  в
прозелени меловых глыб, и вот берег: перламутровая россыпь ракушек,  серая
изломистая кайма нацелованной волнами гальки и дальше -  перекипающее  под
ветром вороненой рябью стремя Дона. На  восток,  за  красноталом  гуменных
плетней,  -  Гетманский  шлях,  полынная  проседь,  истоптанный   конскими
копытами бурый, живущий придорожник,  часовенка  на  развилке;  за  ней  -
задернутая текучим маревом степь. С юга - меловая хребтина горы. На  запад
- улица, пронизывающая площадь, бегущая к займищу.
   В предпоследнюю  турецкую  кампанию  вернулся  в  хутор  казак  Мелехов
Прокофий. Из Туретчины привел он  жену  -  маленькую,  закутанную  в  шаль
женщину. Она прятала лицо, редко показывая тоскующие одичалые глаза. Пахла
шелковая шаль далекими неведомыми запахами, радужные узоры ее питали бабью
зависть. Пленная турчанка сторо

# Часть 2
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:  
подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;  
проведите тестирование аналогично п.1, но при помощи биграмм.

In [7]:
freq_dict_bi_anna = freq_dict_create(anna_karenina,bigrams=True)
freq_dict_bi_war = freq_dict_create(war_peace,bigrams=True)
freq_dict_bi_war_en = freq_dict_create(war_peace_en,bigrams=True)

In [8]:
print("Оригинальный текст:\n",
      test_text, '\n\n'
      "Закодированый текст:\n",
      encoder(test_text,bigrams=True), '\n\n'
      "Декодированый текст:\n",
      decoder(encoder(test_text), freq_dict_bi_anna,True), '\n\n',
      "Оценка:\n",
      sum(i == j for i,j in zip(re.sub(r'[^А-Яа-я ]', '', test_text.lower()), decoder(encoder(test_text,True), freq_dict_bi_anna,True)))/ len(test_text)
     )

Оригинальный текст:
 Мелеховский двор - на самом краю хутора.  Воротца  со  скотиньего  база
ведут на  север  к  Дону.  Крутой  восьмисаженный  спуск  меж  замшелых  в
прозелени меловых глыб, и вот берег: перламутровая россыпь ракушек,  серая
изломистая кайма нацелованной волнами гальки и дальше -  перекипающее  под
ветром вороненой рябью стремя Дона. На  восток,  за  красноталом  гуменных
плетней,  -  Гетманский  шлях,  полынная  проседь,  истоптанный   конскими
копытами бурый, живущий придорожник,  часовенка  на  развилке;  за  ней  -
задернутая текучим маревом степь. С юга - меловая хребтина горы. На  запад
- улица, пронизывающая площадь, бегущая к займищу.
   В предпоследнюю  турецкую  кампанию  вернулся  в  хутор  казак  Мелехов
Прокофий. Из Туретчины привел он  жену  -  маленькую,  закутанную  в  шаль
женщину. Она прятала лицо, редко показывая тоскующие одичалые глаза. Пахла
шелковая шаль далекими неведомыми запахами, радужные узоры ее питали бабью
зависть. Пленная турчанка сторо

# Часть 3
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:  
предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;  
реализуйте и протестируйте его, убедитесь, что результаты улучшились.  



In [4]:
def log_likehood_calc(dd,mt, permutation,text,alphas):
    text = text.translate(str.maketrans(alphas, ''.join(permutation)))
    return sum([mt[dd[text[i]], dd[text[i+1]]] for i in range(len(text) - 1)])

In [5]:
alphas = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [6]:
def mcmc(text,iters,alphas):
    dd = {c: i for i, c in enumerate(alphas)}
    transformation_matrix = np.zeros((len(dd), len(dd)))
    clear_text = re.sub(r'[^А-Яа-я ]', '', text.lower())
    ttext = re.sub(r'[^А-Яа-я ]', '', (war_peace + anna_karenina).lower())
    print(dd, 'щ' in ttext)
    for i in range(len(ttext)-1):
        transformation_matrix[dd[ttext[i]], dd[ttext[i+1]]] += 1
    transformation_matrix = np.clip(transformation_matrix, 1,None)
    transformation_matrix = (np.log(transformation_matrix).T - np.log(transformation_matrix.sum(axis=1))).T

    permutation = np.array(list(alphas))
    random.shuffle(permutation)
    clear_text = clear_text.translate(str.maketrans(alphas, ''.join(permutation)))
    log_likelihood = log_likehood_calc(dd,transformation_matrix, permutation, clear_text, alphas)
    best_likelihood = log_likelihood
    permutation_best = permutation.copy()
    for i in tqdm(range(iters)):
        tmp = random.sample(range(len(alphas)), 2)
        permutation[tmp[0]], permutation[tmp[1]] = permutation[tmp[1]], permutation[tmp[0]]
        new_likelihood = log_likehood_calc(dd,transformation_matrix, permutation,clear_text, alphas)
        if new_likelihood >= log_likelihood:
                log_likelihood = new_likelihood
                if new_likelihood > best_likelihood:
                    best_likelihood = new_likelihood
                    permutation_best = permutation.copy()
        else:
            if random.random() < np.exp(new_likelihood - log_likelihood):
                log_likelihood = new_likelihood
            else:
                permutation[tmp[0]], permutation[tmp[1]] = permutation[tmp[1]], permutation[tmp[0]]
    return clear_text.translate(str.maketrans(alphas, ''.join(permutation_best)))

In [42]:
decoded = mcmc(test_text,200000,alphas)

100%|██████████| 200000/200000 [03:47<00:00, 880.74it/s]


In [43]:
print("Оригинальный текст:\n",
      test_text, '\n\n'
      "Декодированый текст:\n",
      decoded, '\n\n',
      "Оценка:\n",
      sum(i == j for i,j in zip(re.sub(r'[^А-Яа-я ]', '', test_text.lower()), decoded))/ len(test_text)
     )

Оригинальный текст:
 Мелеховский двор - на самом краю хутора.  Воротца  со  скотиньего  база
ведут на  север  к  Дону.  Крутой  восьмисаженный  спуск  меж  замшелых  в
прозелени меловых глыб, и вот берег: перламутровая россыпь ракушек,  серая
изломистая кайма нацелованной волнами гальки и дальше -  перекипающее  под
ветром вороненой рябью стремя Дона. На  восток,  за  красноталом  гуменных
плетней,  -  Гетманский  шлях,  полынная  проседь,  истоптанный   конскими
копытами бурый, живущий придорожник,  часовенка  на  развилке;  за  ней  -
задернутая текучим маревом степь. С юга - меловая хребтина горы. На  запад
- улица, пронизывающая площадь, бегущая к займищу.
   В предпоследнюю  турецкую  кампанию  вернулся  в  хутор  казак  Мелехов
Прокофий. Из Туретчины привел он  жену  -  маленькую,  закутанную  в  шаль
женщину. Она прятала лицо, редко показывая тоскующие одичалые глаза. Пахла
шелковая шаль далекими неведомыми запахами, радужные узоры ее питали бабью
зависть. Пленная турчанка сторо

# Часть 4
Расшифруйте сообщение:  
"დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [7]:
krakozyabra = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"

In [ ]:
freq_dict = freq_dict_create(anna_karenina + war_peace)
rus_top_frequent_chars = ''.join(freq_dict)[:len(set(krakozyabra))]
top_messega_chars = ''.join(set(krakozyabra))
message = krakozyabra.translate(str.maketrans(top_messega_chars, rus_top_frequent_chars))

In [90]:
mcmc(message,300000, ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя')

100%|██████████| 300000/300000 [00:35<00:00, 8423.11it/s]


'ндов зь звывен слапройсью вов блиев слапройсью енуде м челял дллктнсвх улелаью оняул баливерей дуланн зднял зь здн дыноров барзвойсл в бломивен прудвпройсью кроо фр блдоныснн инезнаелн фрырсвн умадр глех улснисл х свинял сн лкнтрж'

In [82]:
mcmc(message,300000, ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя')

100%|██████████| 300000/300000 [00:35<00:00, 8511.16it/s]


'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'